### Introduction
The Amazon Customer Reviews dataset is a valuable resource for sentiment analysis tasks. It contains a vast collection of product reviews from diverse categories, providing a rich and varied source of customer sentiments. With its extensive coverage, this dataset offers insights into the opinions and experiences of Amazon customers. By leveraging the text reviews and associated star ratings, researchers and data analysts can delve into the sentiments expressed by customers and develop models to understand and predict customer satisfaction. This dataset serves as a valuable foundation for studying customer sentiment analysis and exploring the factors that influence customer opinions on various products.

Download the dataset
http://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz

In [28]:
# Read the TSV file into a DataFrame named 'data'
data = pd.read_csv(data, delimiter='\t', quoting=3)
print(data.head())

  marketplace  customer_id       review_id  product_id  product_parent  \
0          US     41409413  R2MTG1GCZLR2DK  B00428R89M       112201306   
1          US     49668221  R2HBOEM8LE9928  B000068O48       734576678   
2          US     12338275  R1P4RW1R9FDPEE  B000GGKOG8       614448099   
3          US     38487968  R1EBPM82ENI67M  B000NU4OTA        72265257   
4          US     23732619  R372S58V6D11AT  B00JOQIO6S       308169188   

                                       product_title product_category  \
0  yoomall 5M Antenna WIFI RP-SMA Female to Male ...      Electronics   
1         Hosa GPM-103 3.5mm TRS to 1/4" TRS Adaptor      Electronics   
2        Channel Master Titan 2 Antenna Preamplifier      Electronics   
3  LIMTECH Wall charger + USB Hotsync & Charging ...      Electronics   
4     Skullcandy Air Raid Portable Bluetooth Speaker      Electronics   

   star_rating  helpful_votes  total_votes vine verified_purchase  \
0            5              0            0    N

Use 10% of dataset

In [7]:
import pandas as pd

# Load the data into a DataFrame
data = pd.read_csv('/content/dataset/amazon_reviews_us_Electronics_v1_00.tsv', delimiter='\t', quoting=3)

# Sample a subset of the data (e.g., 10% of the original dataset)
sampled_data = data.sample(frac=0.1, random_state=42)

# Display the first few rows of the sampled data
print(sampled_data.head())

        marketplace  customer_id       review_id  product_id  product_parent  \
1660889          US     20454938   R5KI6FZDSUK1H  B00171MWSO       939898962   
1354324          US     24741526  R1U6433PQBY12R  B0090Z3SPU       617888253   
1821643          US     29123942  R2B32QV9EJUB8S  9985609034       972383144   
2856884          US     46090876  R35MZC8Q03EMRV  B0019DKOVW       709225090   
232973           US      5471436   R28GAE6RPKTRH  B00CVB12RG       587294791   

                                             product_title product_category  \
1660889  Sony ICFS79W AM/FM/Weather Band Digital Tuner ...      Electronics   
1354324         Bose SoundLink Bluetooth Mobile Speaker II      Electronics   
1821643  Premium 50 Foot High Speed HDMI Cable for your...      Electronics   
2856884   KICKER 08 zKICK Stereo System for Microsoft Zune      Electronics   
232973            Brookstone 2.4GHz Wireless TV Headphones      Electronics   

         star_rating  helpful_votes  total_v

- marketplace: The country code for the marketplace where the review was posted (e.g., "US" for the United States).
- customer_id: The unique identifier of the customer who posted the review.
- review_id: The unique identifier of the review.
- product_id: The unique identifier of the product being reviewed.
- product_parent: The parent product identifier. Products with the same parent are variations of the same product.
- product_title: The title or name of the product being reviewed.
- product_category: The category to which the product belongs (e.g., "Electronics").
- star_rating: The rating given by the customer (ranging from 1 to 5 stars).
- helpful_votes: The number of helpful votes received by the review.
- total_votes: The total number of votes (helpful and unhelpful) received by the review.
- vine: Indicates if the review was written as part of the Vine program (an invitation-only program for trusted reviewers).
- verified_purchase: Indicates if the review was written by a verified purchaser of the product.
- review_headline: The headline or summary of the review.
- review_body: The main content or body of the review.
- review_date: The date when the review was posted.


## Data processing

In [8]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
contractions_map = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have","could've": "could have","couldn't": "could not","didn't": "did not", "doesn't": "does not", "don't": "do not",
    "hadn't": "had not","hasn't": "has not","haven't": "have not", "he'd": "he would", "he'll": "he will","he's": "he is","how'd": "how did","how'll": "how will","how's": "how is",
    "I'd": "I would","I'll": "I will","I'm": "I am","I've": "I have","isn't": "is not","it'd": "it would","it'll": "it will","it's": "it is","let's": "let us","mustn't": "must not",
    "shan't": "shall not","she'd": "she would","she'll": "she will","she's": "she is","should've": "should have","shouldn't": "should not","that's": "that is","there's": "there is",
    "they'd": "they would","they'll": "they will","they're": "they are","they've": "they have","wasn't": "was not","we'd": "we would","we'll": "we will","we're": "we are","we've": "we have",
    "weren't": "were not","what'll": "what will","what're": "what are","what's": "what is","what've": "what have","where's": "where is",
    "who'd": "who would","who'll": "who will","who're": "who are","who's": "who is","who've": "who have","won't": "will not","would've": "would have","wouldn't": "would not","you'd": "you would",
    "you'll": "you will","you're": "you are","you've": "you have"
}


**clean_text** function now includes the expansion of contractions using the `contractions_map` dictionary

In [10]:
# Define a function to clean the review text
def clean_text(text):
    # Expand contractions
    for contraction, expansion in contractions_map.items():
        text = text.replace(contraction, expansion)

    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())
    stopwords_set = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stopwords_set)
    lemmatizer = WordNetLemmatizer()
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    return text

# Apply the clean_text function to the review_text column
sampled_data['review_text'] = sampled_data['review_headline'].fillna('') + ' ' + sampled_data['review_body'].fillna('')
sampled_data['review_text'] = sampled_data['review_text'].map(clean_text)

# Drop the unnecessary columns
sampled_data.drop(columns=['review_headline', 'review_body'], inplace=True)

# Display the first few rows of the processed data
print(sampled_data.head())


        marketplace  customer_id       review_id  product_id  product_parent  \
1660889          US     20454938   R5KI6FZDSUK1H  B00171MWSO       939898962   
1354324          US     24741526  R1U6433PQBY12R  B0090Z3SPU       617888253   
1821643          US     29123942  R2B32QV9EJUB8S  9985609034       972383144   
2856884          US     46090876  R35MZC8Q03EMRV  B0019DKOVW       709225090   
232973           US      5471436   R28GAE6RPKTRH  B00CVB12RG       587294791   

                                             product_title product_category  \
1660889  Sony ICFS79W AM/FM/Weather Band Digital Tuner ...      Electronics   
1354324         Bose SoundLink Bluetooth Mobile Speaker II      Electronics   
1821643  Premium 50 Foot High Speed HDMI Cable for your...      Electronics   
2856884   KICKER 08 zKICK Stereo System for Microsoft Zune      Electronics   
232973            Brookstone 2.4GHz Wireless TV Headphones      Electronics   

         star_rating  helpful_votes  total_v

We have a subset of the data that has been processed according to the defined steps, including the expansion of contractions. The resulting sampled_data DataFrame will contain the relevant columns (star_rating, verified_purchase, review_text) for further analysis.

In [11]:
# Drop the unnecessary columns
sampled_data.drop(columns=['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_date'], inplace=True)

# Display the first few rows of the processed data
print(sampled_data.head())


         star_rating                                        review_text
1660889            4  nice quality good reception use shower bathroo...
1354324            5  satisfaction great deal got expected speaker s...
1821643            5  foot high speed hdmi cable foot high speed hdm...
2856884            5  excellent speaker dock pleased speaker dock so...
232973             5       hearing difficult difficult hearing help lot


### Tokenization
Tokenization is the process of breaking down a text into individual words or tokens, which is an essential step in natural language processing tasks like sentiment analysis.

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
import nltk

# Tokenize the review_text column
sampled_data['tokens'] = sampled_data['review_text'].apply(nltk.word_tokenize)

# Display the first few rows of the processed data
print(sampled_data[['review_text', 'tokens']].head())


                                               review_text  \
1660889  nice quality good reception use shower bathroo...   
1354324  satisfaction great deal got expected speaker s...   
1821643  foot high speed hdmi cable foot high speed hdm...   
2856884  excellent speaker dock pleased speaker dock so...   
232973        hearing difficult difficult hearing help lot   

                                                    tokens  
1660889  [nice, quality, good, reception, use, shower, ...  
1354324  [satisfaction, great, deal, got, expected, spe...  
1821643  [foot, high, speed, hdmi, cable, foot, high, s...  
2856884  [excellent, speaker, dock, pleased, speaker, d...  
232973   [hearing, difficult, difficult, hearing, help,...  


In [12]:
sampled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309387 entries, 1660889 to 1431926
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   star_rating  309387 non-null  int64 
 1   review_text  309387 non-null  object
 2   tokens       309387 non-null  object
dtypes: int64(1), object(2)
memory usage: 9.4+ MB


### Word Embedding
Word embeddings, such as Word2Vec or GloVe, require loading pre-trained models that capture the semantic meaning of words. These models can be quite large in size, and loading them into memory may exceed the available resources.

Instead of using word embeddings,I use alternative text representations that are memory-efficient:

**N-grams**: Instead of considering individual words, you can capture the context by using N-grams (sequences of adjacent words). N-grams can be efficient and provide more contextual information compared to individual words.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

#Convert the tokens column from lists to strings
sampled_data['tokens'] = sampled_data['tokens'].apply(lambda x: ' '.join(x))

# Create an instance of CountVectorizer with N-gram range (e.g., unigrams and bigrams)
ngram_range = (1, 2)  # Adjust the range as needed (e.g., (1, 3) for unigrams, bigrams, and trigrams)
vectorizer = CountVectorizer(ngram_range=ngram_range)

# Fit the vectorizer on the tokens column
vectorizer.fit(sampled_data['tokens'])

# Transform the tokens column into a matrix of N-gram features
ngram_features = vectorizer.transform(sampled_data['tokens'])

# Display the shape of the N-gram feature matrix
print("Shape of N-gram feature matrix:", ngram_features.shape)


Shape of N-gram feature matrix: (309387, 3256942)


In this code, we use the CountVectorizer with the desired N-gram range (e.g., unigrams and bigrams) specified by ngram_range. I could adjust the range to include different combinations of unigrams, bigrams, trigrams, etc., by modifying the ngram_range parameter.

By executing this code, we obtain the N-gram feature matrix (ngram_features), where each row represents a document from the `tokens` column, and each column corresponds to an N-gram feature.

Working with N-grams can significantly increase the dimensionality of  feature space, which may impact memory usage and subsequent modeling steps.

But the shape of N-gram feature matrix, (309387, 3256942), indicates that you have 309,387 samples (rows) and 3,256,942 features (columns) in the matrix. While the number of features is large, it is not uncommon in text-based analysis tasks.

With the N-gram feature matrix, I train simple models to learn patterns and relationships between the features and the corresponding target variable

## Regression Model

In [15]:
from sklearn.model_selection import train_test_split

X = sampled_data['tokens']  # Input features (tokens column)
y = sampled_data['star_rating']  # Target variable (star_rating column)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Convert the tokens into numerical features using a vectorization technique, such as CountVectorizer

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_transformed = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_transformed = vectorizer.transform(X_test)


Build a regression model, such as Linear Regression

In [16]:
from sklearn.linear_model import LinearRegression

# Create an instance of Linear Regression model
regression_model = LinearRegression()

# Fit the model on the training data
regression_model.fit(X_train_transformed, y_train)


LinearRegression()

Evaluate the performance of the regression model

In [17]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on the testing data
y_pred = regression_model.predict(X_test_transformed)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared Score:", r2)


Mean Squared Error: 2.7835421091096673
R-squared Score: -0.44277229814641217


The MSE value of 2.7835 indicates the average squared difference between the predicted and actual star ratings. Lower MSE values indicate better model performance, where values closer to zero indicate a better fit to the data.

The R-squared score of -0.4428 represents the coefficient of determination, which measures the proportion of variance in the target variable (star ratings) that is explained by the regression model. R-squared values range from -∞ to 1, where values closer to 1 indicate a better fit. In your case, a negative R-squared score suggests that the regression model does not fit the data well.

These results indicate that the current regression model may not be performing well in predicting the star ratings.

## Naive Bayes

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Convert the tokenized text to TF-IDF features
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(sampled_data['tokens'])  # Replace 'tokens' with the actual column name from your dataset

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, sampled_data['star_rating'], test_size=0.2, random_state=42)  # Replace 'star_rating' with the actual column name from your dataset

# Train the Naive Bayes classifier
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

# Make predictions on the test set
y_pred = naive_bayes.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.89      0.12      0.21      7167
           2       0.00      0.00      0.00      3642
           3       0.76      0.00      0.01      4839
           4       0.95      0.06      0.11     10700
           5       0.59      1.00      0.74     35530

    accuracy                           0.60     61878
   macro avg       0.64      0.24      0.21     61878
weighted avg       0.67      0.60      0.47     61878



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


`MultinomialNB` is a probabilistic classifier based on the Multinomial distribution, commonly used for text classification tasks. It assumes that the features are generated from a multinomial distribution and calculates the likelihood of each class based on the occurrence frequencies of features.

Class 1 (star rating 1):

Precision: 0.89
Recall: 0.12
F1-score: 0.21
Support: 7167
The precision of 0.89 indicates that when the model predicts class 1, it is correct 89% of the time. However, the recall of 0.12 suggests that the model only identifies 12% of the actual instances of class 1. The low F1-score of 0.21 reflects the trade-off between precision and recall, indicating that the model struggles to accurately classify class 1.

Class 2 (star rating 2):

Precision: 0.00
Recall: 0.00
F1-score: 0.00
Support: 3642
The precision, recall, and F1-score for class 2 are all 0, indicating that the model does not correctly classify any instances of class 2. This suggests that the model fails to capture the patterns and characteristics of class 2.

Class 3 (star rating 3):

Precision: 0.76
Recall: 0.00
F1-score: 0.01
Support: 4839
The precision of 0.76 indicates that the model has moderate accuracy in predicting class 3. However, the recall of 0.00 suggests that the model does not capture any instances of class 3 effectively. The low F1-score of 0.01 further highlights the model's struggles in correctly classifying class 3.

Class 4 (star rating 4):

Precision: 0.95
Recall: 0.06
F1-score: 0.11
Support: 10700
The precision of 0.95 indicates that the model has a high accuracy in predicting class 4. However, the recall of 0.06 suggests that the model only identifies 6% of the actual instances of class 4. The low F1-score of 0.11 reflects the trade-off between precision and recall, indicating that the model struggles to accurately classify class 4.

Class 5 (star rating 5):

Precision: 0.59
Recall: 1.00
F1-score: 0.74
Support: 35530
The precision of 0.59 suggests that the model has moderate accuracy in predicting class 5. The recall of 1.00 indicates that the model captures all instances of class 5 correctly. The high F1-score of 0.74 reflects the model's ability to accurately classify class 5.

Overall, the model achieves an accuracy of 0.60, which means it correctly predicts the star rating for 60% of the instances. The macro-average F1-score is 0.21, indicating the overall effectiveness of the model in correctly classifying the different classes is relatively low. The weighted average F1-score is 0.47, reflecting the trade-off between the performance of the model on different classes weighted by their respective support.

### Apply hyperparameter tuning for the Naive Bayes model


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

# Convert the tokenized text to TF-IDF features
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(sampled_data['tokens'])  # Replace 'tokens' with the actual column name from your dataset

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, sampled_data['star_rating'], test_size=0.2, random_state=42)  # Replace 'star_rating' with the actual column name from your dataset

# Define the parameter grid for grid search
param_grid = {'alpha': [0.1, 0.5, 1.0]}  # Adjust the values as needed

# Create the grid search object
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')

# Perform grid search on the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the Naive Bayes classifier with the best hyperparameters
naive_bayes = MultinomialNB(**best_params)
naive_bayes.fit(X_train, y_train)

# Make predictions on the test set
y_pred = naive_bayes.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.70      0.53      0.60      7167
           2       0.22      0.01      0.02      3642
           3       0.43      0.05      0.09      4839
           4       0.50      0.20      0.29     10700
           5       0.67      0.97      0.79     35530

    accuracy                           0.66     61878
   macro avg       0.50      0.35      0.36     61878
weighted avg       0.60      0.66      0.58     61878



In [25]:
# Test a new text
new_text = "This product is amazing!"
X_new = vectorizer.transform([new_text])
y_pred_new = naive_bayes.predict(X_new)

print("Predicted star rating for the new text:", y_pred_new)


Predicted star rating for the new text: [5]


The overall accuracy has increased to 66%, indicating a better classification performance.
The precision, recall, and F1-score for each class have also improved, especially for classes 1, 3, and 4.
The macro average and weighted average metrics have shown better scores, indicating an overall improvement in the model's performance.

Here's a breakdown of the hyperparameter tuning process:

Parameter Grid: First, a parameter grid is defined, `{'alpha': [0.1, 0.5, 1.0]}`which specifies the hyperparameter values to be explored. In the given example, the parameter grid consists of the alpha parameter for the MultinomialNB model.

Grid Search: The GridSearchCV class from scikit-learn is used to perform grid search. It takes the model (MultinomialNB in this case), the parameter grid, and the number of cross-validation folds as input. `grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')`

<img src = "https://miro.medium.com/v2/resize:fit:786/format:webp/1*PdwlCactbJf8F8C7sP-3gw.png">

Cross-Validation: The grid search applies cross-validation on the training data. It splits the training data into multiple folds and trains/evaluates the model on different combinations of the hyperparameter values. `grid_search.fit(X_train, y_train)`

Performance Evaluation: For each combination of hyperparameters, the model is trained and evaluated using the specified scoring metric (accuracy in this case). The performance metrics are recorded.

`best_params = grid_search.best_params_`

Best Hyperparameters: After completing the grid search, the best hyperparameters are identified based on the highest performance score. The best_params_ attribute of the grid search object provides the optimal hyperparameter values.

- `naive_bayes = MultinomialNB(**best_params)`
- `naive_bayes.fit(X_train, y_train)`

Model Training and Evaluation: Finally, the Naive Bayes model is trained using the best hyperparameters, and its performance is evaluated on the test set `y_pred = naive_bayes.predict(X_test)` Make predictions on the test set
 using classification metrics such as precision, recall, and F1-score.

### Decision Trees
CountVectorizer or TF-IDF Vectorizer to convert the tokenized text into a numerical representation

<img src = "https://miro.medium.com/v2/resize:fit:1400/format:webp/1*RsrKmLuFVZcgZ3Z7sOzGKw.png">

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_vectorized = vectorizer.transform(X_test)

# Create an instance of DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=42)

# Fit the decision tree model on the training data
tree.fit(X_train_vectorized, y_train)

# Make predictions on the test data
y_pred = tree.predict(X_test_vectorized)

# Evaluate the model
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           1       0.56      0.57      0.56      7167
           2       0.26      0.20      0.22      3642
           3       0.31      0.27      0.29      4839
           4       0.38      0.37      0.38     10700
           5       0.76      0.80      0.78     35530

    accuracy                           0.62     61878
   macro avg       0.45      0.44      0.45     61878
weighted avg       0.61      0.62      0.61     61878



The decision tree model with CountVectorizer achieved an accuracy of 0.62 on the test set. Here's a breakdown of the classification metrics:

- For class 1, the model achieved a precision of 0.56, recall of 0.57, and F1-score of 0.56.
- For class 2, the model achieved a precision of 0.26, recall of 0.20, and F1-score of 0.22.
- For class 3, the model achieved a precision of 0.31, recall of 0.27, and F1-score of 0.29.
- For class 4, the model achieved a precision of 0.38, recall of 0.37, and F1-score of 0.38.
- For class 5, the model achieved a precision of 0.76, recall of 0.80, and F1-score of 0.78.

The macro average F1-score is 0.45, indicating the overall performance of the model across all classes. The weighted average F1-score is 0.61, considering the class imbalance in the dataset.

Class imbalance refers to a situation where the distribution of target classes in a dataset is not balanced. In other words, one or more classes may have significantly more or fewer instances compared to the other classes. This can be problematic because machine learning algorithms often assume that the classes are balanced and may be biased towards the majority class.